## This notebook goes over the basic modular components of LangChain like models, prompts, chains, agents and memory

### Loading API keys for OpenAI and SerpAPI (Google Search)

In [25]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']
serp_api_key = os.environ['SERPAPI_API_KEY']

### Initializing LLM Model

In [21]:
from langchain.llms import OpenAI

llm_model = OpenAI(temperature = 0.0) 
#temperature determines how creative the model can get with 0 being the least and 1 being the most
#Setting a high temperature also leads to wrong answer generation

name = llm_model("I want to open a restaurant for Mexican Indian Fusion Food. Suggest a fancy name for this.")
print(name)



"Spice of the Aztecs"


### Simple Sequential Chain of Prompts (1 output only)

In [3]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

prompt_template_name = PromptTemplate(
                            input_variables = ['cuisine'],
                            template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this. Only one name will do.")

name_chain = LLMChain(llm = llm_model, prompt = prompt_template_name)

prompt_template_menu = PromptTemplate(
                            input_variables = ['restaurant_name'],
                            template = "Suggest some menu items for {restaurant_name}. Return it as a comma separated list.")

menu_chain = LLMChain(llm = llm_model, prompt = prompt_template_menu)

In [4]:
final_chain = SimpleSequentialChain(chains = [name_chain, menu_chain])

response = final_chain.run("Mexican")

print(response)

c:\users\nihaa\appdata\local\programs\python\python39\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




1. Tacos al Pastor
2. Enchiladas Verdes
3. Carne Asada Burrito
4. Quesadillas de Pollo
5. Chimichangas
6. Chile Relleno
7. Fajitas de Res
8. Sopa de Tortilla
9. Carnitas Tostadas
10. Nachos Supreme
11. Baja Fish Tacos
12. Veggie Quesadilla
13. Shrimp Ceviche
14. Tamales de Pollo
15. Flautas de Carne.


In [5]:
prompt_template_name = PromptTemplate(
                            input_variables = ['cuisine'],
                            template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this. Only one name will do.")

name_chain = LLMChain(llm = llm_model, prompt = prompt_template_name, output_key = "restaurant_name")

prompt_template_menu = PromptTemplate(
                            input_variables = ['restaurant_name'],
                            template = "Suggest some menu items for {restaurant_name}. Return it as a comma separated list.")

menu_chain = LLMChain(llm = llm_model, prompt = prompt_template_menu, output_key = "menu_items")

### Sequential Chain of prompts (allows multiple outputs)

In [6]:
from langchain.chains import SequentialChain

final_chain = SequentialChain(chains = [name_chain, menu_chain],
                              input_variables = ['cuisine'],
                              output_variables = ['restaurant_name', 'menu_items'])

response = final_chain({'cuisine' : "Greek"})

print(response)

c:\users\nihaa\appdata\local\programs\python\python39\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'cuisine': 'Greek', 'restaurant_name': '\n\n"Olympus Eats"', 'menu_items': "\n\n1. Zeus' Thunder Burger\n2. Athena's Greek Salad\n3. Apollo's Grilled Chicken Gyro\n4. Poseidon's Seafood Platter\n5. Hera's Lamb Souvlaki\n6. Demeter's Veggie Wrap\n7. Dionysus' Wine-Infused Pasta\n8. Hades' Inferno Pizza\n9. Artemis' Wild Game Skewers\n10. Hermes' Mediterranean Flatbread\n11. Aphrodite's Shrimp Scampi\n12. Hephaestus' BBQ Ribs\n13. Ares' Spicy Chicken Wings\n14. Persephone's Quinoa Bowl\n15. Hestia's Fire-Roasted Vegetables."}


### Utilizing wikipedia and math agents

In [19]:
from langchain.agents import AgentType, initialize_agent, load_tools

tools = load_tools(["wikipedia", "llm-math"], llm = llm_model)

agent = initialize_agent(tools, llm_model, agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION) #Think, Reason and Perform Action

agent.run("When was Bill Gates born and how old is he?")

'Bill Gates was born on October 28, 1955 and is currently 67 years old.'

In [22]:
from langchain.agents import AgentType, initialize_agent, load_tools

tools = load_tools(["wikipedia", "llm-math"], llm = llm_model)

agent = initialize_agent(tools, llm_model, agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose = True) #Think, Reason and Perform Action

agent.run("When was Elon Musk born and how old is he?")



> Entering new AgentExecutor chain...
 I should use Wikipedia to find information about Elon Musk
Action: Wikipedia
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect and former chairman of Tesla, Inc.; owner, chairman and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is the wealthiest person in the world, with an estimated net worth of US$232 billion as of December 2023, according to the Bloomberg Billionaires Index, and $254 billion according to Forbes, primarily from his ownership stakes in Tesla and SpaceX.A member of the wealthy South African Musk family, Elon was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mot

Thought: I should use Calculator to calculate Elon Musk's age
Action: Calculator
Action Input: 2022 - 1971
Observation: Answer: 51
Thought: I now know the final answer
Final Answer: Elon Musk was born on June 28, 1971 and is currently 51 years old.

> Finished chain.


'Elon Musk was born on June 28, 1971 and is currently 51 years old.'

### Utilizing Google and math agents

In [39]:
tools = load_tools(["serpapi", "llm-math"], llm = llm_model)

agent = initialize_agent(tools, llm_model, agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose = True) #Think, Reason and Perform Action

agent.run("What is Elon Musks current age? Find out age + 5*23/10?")



> Entering new AgentExecutor chain...
 I should use a calculator to solve this equation.
Action: Calculator
Action Input: 5*23/10
Observation: Answer: 11.5
Thought: Now I need to add this to Elon Musk's current age.
Action: Calculator
Action Input: 11.5 + 49
Observation: Answer: 60.5
Thought: I now know the final answer.
Final Answer: Elon Musk's current age is 60.5 years old.

> Finished chain.


"Elon Musk's current age is 60.5 years old."

### Simulating memory based conversations

In [41]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

memory_chain = LLMChain(llm = llm_model, prompt = prompt_template_name, memory = memory)

name = memory_chain.run("Mexican")

print(name)



"El Jardín de Sabores" (The Garden of Flavors)


In [42]:
name = memory_chain.run("German")

print(name)



"Kaiser's Kitchen"


In [43]:
memory_chain.memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Mexican'), AIMessage(content='\n\n"El Jardín de Sabores" (The Garden of Flavors)'), HumanMessage(content='German'), AIMessage(content='\n\n"Kaiser\'s Kitchen"')]))

In [46]:
print(memory_chain.memory.buffer)

Human: Mexican
AI: 

"El Jardín de Sabores" (The Garden of Flavors)
Human: German
AI: 

"Kaiser's Kitchen"


### Remember only last 3 conversation exchanges to save costs

In [48]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm = llm_model)
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [49]:
convo.run("Who won the first football world cup?")

' The first football world cup was won by Uruguay in 1930. They defeated Argentina 4-2 in the final match. The tournament was held in Uruguay and featured 13 teams from around the world. The winning goal was scored by Uruguayan player Hector Castro.'

In [50]:
convo.run("What is 10+10")

' 10+10 is equal to 20.'

In [51]:
convo.run("Who was the captain of the losing team?")

' The captain of the losing team in the first football world cup was Manuel Ferreira of Argentina. He was a defender and played for the club team Boca Juniors.'

In [54]:
print(convo.memory.buffer)

Human: Who won the first football world cup?
AI:  The first football world cup was won by Uruguay in 1930. They defeated Argentina 4-2 in the final match. The tournament was held in Uruguay and featured 13 teams from around the world. The winning goal was scored by Uruguayan player Hector Castro.
Human: What is 10+10
AI:  10+10 is equal to 20.
Human: Who was the captain of the losing team?
AI:  The captain of the losing team in the first football world cup was Manuel Ferreira of Argentina. He was a defender and played for the club team Boca Juniors.


In [63]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k = 1)

convo = ConversationChain(llm = llm_model, memory = memory)

convo.run("Who won the first football world cup?")

' The first football world cup was won by Uruguay in 1930. They defeated Argentina 4-2 in the final match. The tournament was held in Uruguay and featured 13 teams from around the world. The winning goal was scored by Uruguayan player Hector Castro.'

In [64]:
convo.run("What is 10+10")

' 10+10 is equal to 20.'

In [65]:
convo.run("What is 20+20?")

' 20+20 is equal to 40.'

In [66]:
convo.run("Who was the captain of the losing team?")

' I do not have enough context to accurately answer that question. Can you provide more information?'

In [67]:
print(convo.memory.buffer)

Human: Who was the captain of the losing team?
AI:  I do not have enough context to accurately answer that question. Can you provide more information?


### Setting memory to 3 and trying again

In [68]:
memory = ConversationBufferWindowMemory(k = 3)

convo = ConversationChain(llm = llm_model, memory = memory)

convo.run("Who won the first football world cup?")

' The first football world cup was won by Uruguay in 1930. They defeated Argentina 4-2 in the final match. The tournament was held in Uruguay and featured 13 teams from around the world. The winning goal was scored by Uruguayan player Hector Castro.'

In [69]:
convo.run("What is 10+10")

' 10+10 is equal to 20.'

In [70]:
convo.run("Who was the captain of the winning team?")

' The captain of the winning team, Uruguay, was José Nasazzi. He was a defender and played for the club Nacional in Uruguay. He also played for the Uruguayan national team and was known for his strong leadership skills on the field.'